In [45]:
import requests, time
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# service = Service('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')
# driver = webdriver.Chrome(service=service)
# url= 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
# driver.get(url)

In [50]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
base_url = 'https://www.ggpolice.go.kr'
main_url = '/main/html.do?menu=HC44'
url = f'{base_url}{main_url}'
res = requests.get(url,headers=header)
soup = BeautifulSoup(res.text)

In [51]:
lis = soup.select('.road')
len(lis)

31

In [52]:
li = lis[0]

In [53]:
href = li.select_one('.road > a')['href']
href = href[26:]
href

'/swjb/html.do?menu=HC24'

In [54]:
sub_url = f'{base_url}{href}'
res = requests.get(sub_url)
caps_soup = BeautifulSoup(res.text)

In [55]:
addr = caps_soup.select('.basic_list7 > li')[1].get_text()[8:-8].strip()
addr

'경기도 수원시 장안구 정자천로 199'

In [56]:
name = caps_soup.select('.basic_list7 > li')[0].get_text()[6:].strip()
name

'수원중부경찰서'

In [63]:
lines = []
for li in tqdm(lis):
    try:
        href = li.select_one('.road > a')['href']
        href = href[26:]
        sub_url = f'{base_url}{href}'
        res = requests.get(sub_url)
        caps_soup = BeautifulSoup(res.text)
        name = caps_soup.select('.basic_list7 > li')[0].get_text()[6:].strip()
        addr = caps_soup.select('.basic_list7 > li')[1].get_text()[8:-8].strip()
        lines.append({'기관명':name,'주소':addr})
    except:
        continue

100%|██████████| 31/31 [00:18<00:00,  1.71it/s]


In [67]:
df = pd.DataFrame(lines)
df.head(30).tail(10)

,기관명,주소
19,화성서부경찰서,경기 화성시 남양읍 남양로 570
20,용인동부경찰서,경기 용인시 처인구 금학로 143
21,수원남부경찰서,경기 수원시 영통구 매봉로 52
22,광주경찰서,경기 광주시 포돌이로 135
23,이천경찰서,경기 이천시 부악로 32
24,안성경찰서,경기 안성시 알미산로 140
25,여주경찰서,경기 여주시 세종로 50
26,양평경찰서,경기도 양평군 옥천면 고읍로 8
27,의왕경찰서,경기 의왕시 오봉로 10
28,하남경찰서,경기 하남시 검단로 27


In [68]:
df.to_csv('data/caps_addr.csv',index=False)
pd.read_csv('data/caps_addr.csv').tail()

,기관명,주소
24,안성경찰서,경기 안성시 알미산로 140
25,여주경찰서,경기 여주시 세종로 50
26,양평경찰서,경기도 양평군 옥천면 고읍로 8
27,의왕경찰서,경기 의왕시 오봉로 10
28,하남경찰서,경기 하남시 검단로 27


In [69]:
lines = []
for li in tqdm(lis):
    try:
        href = li.select_one('.road > a')['href']
        href = href[26:]
        sub_url = f'{base_url}{href}'
        res = requests.get(sub_url)
        caps_soup = BeautifulSoup(res.text)
        name = caps_soup.select('.basic_list7 > li')[0].get_text()[6:].strip()
        addr = caps_soup.select('.basic_list7 > li')[1].get_text()[8:-8].strip()
        if not any(line['기관명'] == name for line in lines):
            lines.append({'기관명':name,'주소':addr})
    except:
        continue

100%|██████████| 31/31 [00:18<00:00,  1.68it/s]


In [70]:
df2 = pd.DataFrame(lines)
df2.tail()

,기관명,주소
23,안성경찰서,경기 안성시 알미산로 140
24,여주경찰서,경기 여주시 세종로 50
25,양평경찰서,경기도 양평군 옥천면 고읍로 8
26,의왕경찰서,경기 의왕시 오봉로 10
27,하남경찰서,경기 하남시 검단로 27


In [71]:
df2.to_csv('data/caps_addr.csv',index=False)
pd.read_csv('data/caps_addr.csv').tail()

,기관명,주소
23,안성경찰서,경기 안성시 알미산로 140
24,여주경찰서,경기 여주시 세종로 50
25,양평경찰서,경기도 양평군 옥천면 고읍로 8
26,의왕경찰서,경기 의왕시 오봉로 10
27,하남경찰서,경기 하남시 검단로 27


In [72]:
with open('../04.지도시각화/data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [73]:
base_url = "https://dapi.kakao.com/v2/local/search/address.json"
addr = ''
url = f'{base_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query='

In [77]:
k_header = {'Authorization':f'KakaoAK {kakao_key}'}

In [86]:
lat_list = []
lng_list = []
for i in tqdm(df2.index):
    try:
        url = f'{base_url}?query={quote(df2.주소[i])}'
        result = requests.get(url, headers=k_header).json()
        lat_list.append(float(result['documents'][0]['y']))
        lng_list.append(float(result['documents'][0]['x']))
    except:
        print(df2.기관명[i], df2.주소[i])

 68%|██████▊   | 19/28 [00:06<00:03,  2.89it/s]

화성동탄경찰서 경기도 화성시 동탄대로 13길 7


 93%|█████████▎| 26/28 [00:09<00:00,  2.88it/s]

양평경찰서 경기도 양평군 옥천면 고읍로 8


100%|██████████| 28/28 [00:09<00:00,  2.86it/s]


In [81]:
df2['위도'] = lat_list[]
df2['경도'] = lng_list[]
df2.head()


ValueError: Length of values (24) does not match length of index (28)